In [1]:
import warnings
import os
os.environ["OMP_NUM_THREADS"] = "6"
# warnings.filterwarnings("ignore", message="KMeans is known to have a memory leak*")
# warnings.filterwarnings("ignore", message="Solution may be inaccurate*")

In [2]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

import stratifreg
from stratifreg.two_groups import Joint2Regressor
from stratifreg.k_groups import JointKRegressor
from stratifreg.gmm_groups import Joint2GMMRegressor
from stratifreg.utils import JointUtils

In [3]:
def polynomial_tramsform(X, degree=2, include_bias=False):
    """
    Transform a DataFrame Pandas/Array Numpy [ X (n, p) ] with polynoms 
    Example : degree = 2 -> 1, x1, x2, ..., x1^2, x2^2, x1*x2, etc.
    """
    poly = PolynomialFeatures(degree=degree, include_bias=include_bias)
    X_poly       = poly.fit_transform(X)
    colname_poly = poly.get_feature_names_out(X.columns)
    X_poly_df = pd.DataFrame(X_poly, columns=colname_poly, index=X.index)
    return X_poly_df, colname_poly

def get_data(path_X,path_y,scale=False,degree=0):
    scaler = StandardScaler()
    X = pd.read_csv(path_X)
    y = pd.read_csv(path_y)
    columnsXpoly = X.columns
    if degree>1: 
        X,columnsXpoly = polynomial_tramsform(X,degree,False)
    X  = pd.DataFrame(scaler.fit_transform(X), columns=columnsXpoly)
    X1,X2,y1,y2  = JointUtils.split_at_y0(X,y)
    if scale: X = scaler.fit_transform(X)
    if scale: X1 = scaler.fit_transform(X1)
    if scale: X2 = scaler.fit_transform(X2)
    Xc          = JointUtils.add_intercept(X)
    X1c         = JointUtils.add_intercept(X1)
    X2c         = JointUtils.add_intercept(X2)
    return X,Xc,y,X1,X1c,y1,X2,X2c,y2,columnsXpoly

X,Xc,y,X1,X1c,y1,X2,X2c,y2,varnames = get_data('./datasets/Xf_all_datasurvey.csv',
                                               './datasets/yf_all_datasurvey.csv')
print(Xc.shape,X1c.shape,X2c.shape,y.shape,y1.shape,y2.shape)

(4361, 7) (2231, 7) (2130, 7) (4361, 1) (2231, 1) (2130, 1)


In [4]:
print(X.shape, X1c.shape,X2c.shape)
print(X.head())

(4361, 6) (2231, 7) (2130, 7)
      hstrs    financ      fear     angry     happy      bord
0  0.934975 -1.178120  0.664019 -0.849668  0.768101  0.102943
1  1.654081 -0.826310  0.491007  0.141794 -3.110656  0.630884
2  1.341426  0.009237  0.491007  0.141794 -0.138151  0.289275
3 -0.940954  0.229118  0.620766  0.586243 -0.790652  0.692994
4  0.090807  0.844784  0.491007  0.141794 -0.138151  0.289275


In [5]:
reg1 = Joint2Regressor()
[beta], var_beta, sigma2s = reg1.fit_ols_single(Xc, y)
print(JointKRegressor.display(reg1, "beta"))
print(np.round([reg1.variables_["lgk"],reg1.variables_["bic"],reg1.variables_["aic"]],2))

        beta_G1
const   26.9424
hstrs    2.6833
financ   1.6128
fear    -2.4335
angry   -2.5095
happy    5.7974
bord   -12.9043
[-20886.83  41840.7   41789.66]


In [6]:
reg = Joint2Regressor()
[beta1, beta2], sigma2s = reg.fit_ols_groups(X1c, X2c, y1, y2, sigma_mode='two')
print(JointKRegressor.display(reg, "beta"))
print(np.round([reg.lgk1_,reg.bic1_,reg.aic1_],2))
print(np.round([reg.lgk2_,reg.bic2_,reg.aic2_],2))

        beta_G1  beta_G2
const    1.0844  51.4664
hstrs   -1.0810   1.6608
financ   0.3151   1.4477
fear     0.3509  -1.5188
angry   -1.2632  -2.0165
happy    1.6020   3.8584
bord    -3.6285  -5.5448
[-9356.11 18773.9  18728.22]
[-9461.39 18984.1  18938.79]


In [7]:
kreg = JointKRegressor()
rr = kreg.fit([(X1c,y1),(X2c,y2)], joint_X_list=None, loss='quadratic', 
         tau=0.0, l1=0., l2=0., weights_list=None)
print(kreg.X_columns_,len(kreg.X_columns_),len(rr[0]))
print(JointKRegressor.display(kreg, "beta"))

['const', 'hstrs', 'financ', 'fear', 'angry', 'happy', 'bord'] 7 7
        beta_G1  beta_G2
const    1.0844  51.4664
hstrs   -1.0810   1.6608
financ   0.3151   1.4477
fear     0.3509  -1.5188
angry   -1.2632  -2.0165
happy    1.6020   3.8584
bord    -3.6285  -5.5448


In [8]:
gmmreg = Joint2GMMRegressor()
gmmreg.fit(X1c, X2c, y1, y2, x0=None, m1=1, m2=1, max_iter=10)
print(Joint2GMMRegressor.display(gmmreg, "beta"))
# print(Joint2GMMRegressor.predict(gmmreg, Xc,"beta")[:5])

        beta_G1_C1  beta_G2_C1
const       1.0844     51.4662
hstrs      -1.0810      1.6609
financ      0.3151      1.4477
fear        0.3509     -1.5188
angry      -1.2632     -2.0165
happy       1.6020      3.8584
bord       -3.6285     -5.5448


In [9]:
kreg = JointKRegressor()
kreg.fit([(X1c,y1),(X2c,y2)], joint_X_list=None, loss='quantile', 
                   tau=0.5, l1=0., l2=0., weights_list=None)
print(JointKRegressor.display(kreg, "beta"))

        beta_G1  beta_G2
const    0.7805  48.2192
hstrs   -0.0210   1.8404
financ   0.0024   2.6541
fear     0.0058  -1.9495
angry   -0.0138  -2.5632
happy    0.3363   4.8254
bord    -0.7847  -7.0348


C:\Users\prima\anaconda3\Lib\site-packages\cvxpy\problems\problem.py:1504: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [10]:
x0_c  = JointUtils.find_x0(Xc, y)
x0_LL = JointUtils.find_x0_LL(Xc, y, L=1)
regctr_x0c  = Joint2Regressor()
resu_x0c    = regctr_x0c.fit_ols_jointure_a_b(X1c, X2c, y1, y2, x0_c, 
                                  y0=None, sigma_mode='one', cas='a')
regctr_x0LL = Joint2Regressor()
resu_x0LL   = regctr_x0LL.fit_ols_jointure_a_b(X1c, X2c, y1, y2, x0_LL, 
                                  y0=None, sigma_mode='one', cas='a')

print(regctr_x0c.X_columns_, regctr_x0LL.X_columns_)
print(JointKRegressor.display(regctr_x0c, "beta_x0_c"))
JointUtils.check_jointure_constraint(resu_x0c[0],[x0_c])
print(JointKRegressor.display(regctr_x0LL, "beta_x0_LL"))
JointUtils.check_jointure_constraint(resu_x0LL[0],[x0_LL])
# Joint2Regressor.predict(regctr_x0c,Xc,1)[:5]

['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7'] ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7']
    beta_x0_c_G1  beta_x0_c_G2
x1        7.0393       46.3565
x2       -5.1902        7.0177
x3        4.3100       -3.2720
x4       -3.3767        0.5521
x5        5.8358       -7.9274
x6       10.0764       -0.9389
x7       -5.6051       -6.5736
Joint 1: left=29.507377, right=29.507377, diff=1.07e-14  (constraint OK)  (   )
    beta_x0_LL_G1  beta_x0_LL_G2
x1         7.6600        46.3337
x2         3.8592        -2.6193
x3        -5.0614         8.0289
x4         5.5954        -6.4034
x5        -4.5915         0.6121
x6         6.9564         1.2443
x7        -4.8048        -8.8670
Joint 1: left=29.696501, right=29.696501, diff=3.55e-15  (constraint OK)  (   )


In [11]:
gmmreg = Joint2GMMRegressor()
beta_mat, sigma2_1, sigma2_2 = gmmreg.fit(X1c, X2c, y1, y2, x0=x0_c, m1=1, m2=1,max_iter=10)
print(Joint2GMMRegressor.display(gmmreg,"beta"))

        beta_G1_C1  beta_G2_C1
const       7.0393     46.3562
hstrs      -5.1902      7.0177
financ      4.3100     -3.2720
fear       -3.3767      0.5521
angry       5.8357     -7.9274
happy      10.0763     -0.9389
bord       -5.6050     -6.5736


In [12]:
gmmreg = Joint2GMMRegressor()
gmmreg.fit(X1c, X2c, y1, y2, x0=x0_c, m1=2, m2=2,max_iter=10)
print(Joint2GMMRegressor.display(gmmreg,"beta"))
Joint2GMMRegressor.check_jointure_constraint(gmmreg,x0_c,2,True)

        beta_G1_C1  beta_G1_C2  beta_G2_C1  beta_G2_C2
const       7.6490      4.4406     46.0123     47.2639
hstrs      -4.2729     -5.8737      7.7101      5.4304
financ      3.0442      6.0747     -5.0522     -1.3369
fear       -2.8366     -4.4864      4.5111     -1.4011
angry       0.1446     29.4225    -10.2859     -5.5235
happy       7.7955     20.1991     -9.8568      4.7118
bord       -6.8140     -1.5245    -10.6694     -4.3168
Joint: left=31.503881, right=31.503881, diff=0.00e+00 (constraint OK)


(0.0, True)

In [13]:
kreg = JointKRegressor()
group_list = [(X1c,y1),(X2c,y2)]
kreg.fit(group_list, joint_X_list=None, loss='quadratic', 
         tau=0.5, l1=0.0, l2=0.0, weights_list=None)
print(JointKRegressor.display(kreg,"beta"))

        beta_G1  beta_G2
const    1.0844  51.4664
hstrs   -1.0810   1.6608
financ   0.3151   1.4477
fear     0.3509  -1.5188
angry   -1.2632  -2.0165
happy    1.6020   3.8584
bord    -3.6285  -5.5448


In [14]:
kreg = JointKRegressor()
betas = kreg.fit([(X1c,y1),(X2c,y2)], joint_X_list=[x0_c], loss='quadratic', 
         tau=0.5, l1=0.0, l2=0.0, weights_list=None) 
print(JointKRegressor.display(kreg,"beta"))
JointUtils.check_jointure_constraint(betas,[x0_c],tol=1e-3)

        beta_G1  beta_G2
const    7.0393  46.3565
hstrs   -5.1902   7.0177
financ   4.3100  -3.2720
fear    -3.3767   0.5521
angry    5.8358  -7.9274
happy   10.0764  -0.9389
bord    -5.6051  -6.5736
Joint 1: left=29.507377, right=29.507377, diff=1.54e-07  (constraint OK)  (   )


In [15]:
kreg = JointKRegressor()
X2c1,X2c2,y21,y22  = JointUtils.split_at_y0(X2c,y2)
jl = [x0_c,JointUtils.find_x0(X2c, y2)]

betas = kreg.fit([(X1c,y1),(X2c1,y21),(X2c2,y22)], jl, loss='quantile', 
                 tau=0.5, l1=0.9, l2=0.3)
print(JointKRegressor.display(kreg,"beta"))
# JointKRegressor.predict(kreg,Xc,1)[:5]

        beta_G1  beta_G2  beta_G3
const    4.1740  32.4177  64.8290
hstrs   -1.1739   3.6166   3.0070
financ   0.7411  -3.6753   0.4303
fear    -1.4684   2.9758  -2.6811
angry    1.8887 -10.1307   1.8236
happy    3.5840  -7.1827   5.3531
bord    -3.2905  -2.3687  -3.3200


In [16]:
import statsmodels.api as sm
model  = sm.OLS(y, Xc.values).fit()
model1 = sm.OLS(y1, X1c.values).fit()
model2 = sm.OLS(y2, X2c.values).fit()
print(np.round([model.llf, model.bic, model.aic],2))
print(np.round([model1.llf, model1.bic, model1.aic],2))
print(np.round([model2.llf, model2.bic, model2.aic],2))

[-20886.83  41832.32  41787.66]
[-9356.11 18766.19 18726.22]
[-9461.39 18976.43 18936.79]
